Goal is to translate data from Pixelav (which comes in the form of 8x32 array) into something the board can understand (string of bits 00, 01, 10, 11 that corresponds to different charge thresholds). Then want to map it into array2.
\
pixelav: 0:3 y, 0:15 x (8x32 --> 256 pixels). then each pixel goes to 2 bits which classify its charge.  
\
We realized that the current pixelav output is actually **13x21**, not **8x32**, so we are truncating values from the top and bottom rows of the original shape and padding with zeroes where we need in order to reshape the array.  
\
I'm currently using example parquet files form Eliza in order to structure this script. Thse files have 28127 clusters, each with 20 13x21 arrays (one every 200ps until 4ns).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

In [ ]:
final_size = (8,32)
thresh0 = 400
thresh1 = 1600
thresh2 = 2400

In [ ]:
def reshape_pixelav(cluster, cluster_id, time_id):
    """
    Inputs: 13x21 array from Pixelav, labelled with the time slice
    Outputs: 8x32 array with (hopefully) the same data which can be fed into Array2
    
    Truncates original rows on top and bottom and adds zeros to the extra columns on the right in order to fit 8x32 shape. 
    To avoid accidentally truncating data, we add the values of each deleted pixel to the nearest row. For example, since 
    cut off row 12, if there were any charge in that row it would be added to the same column in row 9, which is the last
    column we save. A message will appear if this happens.
    """
    # truncating and overflow
    pixelav_row = np.shape(cluster)[0]
    pixelav_col = np.shape(cluster)[1]
    chop_start = 1
    chop_stop = 9
    overflow_top = np.zeros(pixelav_col)
    overflow_bottom = np.zeros(pixelav_col)
    for col in range(pixelav_col):
        for row in range(chop_start):
            overflow_top[col] += cluster[row, col]
        for row in range(chop_stop + 1, pixelav_row):
            overflow_bottom[col] += cluster[row, col]
    cluster[chop_start, :] += overflow_top
    cluster[chop_stop, :] += overflow_bottom
    chop_cluster = cluster[chop_start:chop_stop, :]

    if np.any(overflow_top) or np.any(overflow_bottom):
        print(f"Overflow detected in cluster {cluster_id} at {time_id}ps:")
        if np.any(overflow_top):
            print(f"  Top overflow: {overflow_top}")
        if np.any(overflow_bottom):
            print(f"  Bottom overflow: {overflow_bottom}")     

    #padding
    final_input = np.zeros(final_size)
    final_input[:, :chop_cluster.shape[1]] = chop_cluster
    return final_input

In [ ]:
def make_bit_string(final_input, reshape=False, cluster=None, cluster_id=None, time_id=None, image_cluster=True):
    """
    Inputs: Reshaped 8x32 array of Pixelav data
    Outputs: String of bits in (hopefully) correct order (string should start with LSB, going RT-RB-LT-LB and end with MSB), 
    ready to go into Array 2
    """
    if reshape:
        final_input = reshape_pixelav(cluster, cluster_id, time_id)
    
    big_row_start= 0
    pixel_data = []
    heatmap = np.zeros((8,32))
    while big_row_start < 8:
        for column in reversed(range(32)):
            for mini_row in range(2):
                row = mini_row + big_row_start
                charge = final_input[row, column]
                if charge <= thresh0: 
                    label = "00"
                    colorval = 0
                elif thresh0 < charge < thresh1:
                    label = "01"
                    colorval = 1
                elif thresh1 < charge < thresh2:
                    label = "10"
                    colorval = 2
                else:
                    label = "11"
                    colorval = 3
                pixel_data.append(label)
                heatmap[row, column] = colorval
        big_row_start += 2
    bit_string = ''.join(pixel_data)

    if image_cluster:
        fig, ax = plt.subplots()
        cax = ax.imshow(heatmap, cmap='viridis', interpolation='none') 
        plt.colorbar(cax, label="Bit Label Value (00 -> 11)")
        
        ax.set_xticks(np.arange(-0.5, 32, 1), minor=True)
        ax.set_yticks(np.arange(-0.5, 8, 1), minor=True)
        ax.grid(which='minor', color='w', linestyle='-', linewidth=2)
        ax.tick_params(which='minor', size=0)

        plt.show()
    return bit_string

In [ ]:
ar2row = 256
ar2col = 16
array2 = np.zeros((ar2row,ar2col))
len_w2_f2 = (104*16)+8
len_bits = (8*32)*2

# fake numbers for shift register and dnn parts of the array
w2_f2 = [0]*len_w2_f2
unused = [0]*((256*16)-len_w2_f2-len_bits)

def make_array2(bit_string):
    i=0
    for row in range(ar2row):
        for col in reversed(range(ar2col)):
            if i < len_w2_f2:
                array2[row,col] = w2_f2[i]
            elif len_w2_f2 <= i < len_w2_f2 + len_bits:
                array2[row, col] = int(bit_string[i - len_w2_f2])
            elif i >= len_w2_f2+len_bits:
                array2[row, col] = unused[i - (len_w2_f2+len_bits)]
            i += 1
    return array2

In [ ]:
# using eliza's BIB parquet file as a sample pixelav output
truth = pd.read_parquet('labelsBIBtracks.parquet')
recon2d = pd.read_parquet('recon2DBIBtracks.parquet')
recon3d = pd.read_parquet('recon3DBIBtracks.parquet')
print(truth.shape, recon2d.shape, recon3d.shape)

In [ ]:
# i'm currently just doing a few particle clusters because there are a lot, but if you want to do all just 
# switch which "for cluster_id" line is uncommented
array2_dict = {}
#for cluster_id in range(recon3d.shape[0]):
for cluster_id in range(5, 10):
    array2_dict[cluster_id] = []
    for time_idx in range(20):
        clusterSlice = recon3d.iloc[cluster_id].to_numpy().reshape(20,13,21)[time_idx]
        time_label = 200*time_idx + 200
        final_in = reshape_pixelav(clusterSlice, cluster_id, time_label) 
        bit_string = make_bit_string(final_in)
        array = make_array2(bit_string)
        array2_dict[cluster_id].append(array)

**Scan chain to Shift Register conversion:**  
The Shift Register data is a series of 2-bit representations per pixel that starts in the top right corner of the array (figure 5) and goes RT-RB-LT-LB. Meanwhile, the Scan Chain takes 3 bits per pixel (will add this transition soon), starts with the right side of the array (pixel 63), and goes in a LB-RB-RT-LT order. This code will translate between those two formats so that we can tell which spot in the array is coming from which pixel. 
 

In [ ]:
test_string = [0]*512
test_string[0], test_string[-1] = 1, 1
test_string = ''.join(map(str, test_string))

def shiftreg_to_scanchain(bit_string):
    big_row_len = 16*2*2*2
    row_len = int(big_row_len/2)
    big_row_start = 0
    scanchain_map = []
    while big_row_start < big_row_len*4:
        section_row0 = bit_string[big_row_start:(big_row_start + row_len)][::-1]
        section_row1 = bit_string[big_row_start+row_len:big_row_start+row_len*2][::-1]

        move_right = 0
        while move_right < 64:
            idx = 0 + move_right
            lb = section_row1[idx] + section_row1[idx+1]
            rb = section_row1[idx+2] + section_row1[idx+3]
            rt = section_row0[idx+2] + section_row0[idx+3]
            lt = section_row0[idx] + section_row0[idx+1]

            square = lb + rb + rt + lt
            scanchain_map.append(square)
            move_right += 4
        big_row_start += big_row_len
    scanchain_map = ''.join(scanchain_map)
    
    # assuming thresholds are the same as in the 2-bit version 
    scanchain_map_final = []
    for i in range(0, len(scanchain_map), 2):
        pixel = scanchain_map[i]+scanchain_map[i+1]
        if pixel == '00':
            newpixel = '000'
        elif pixel == '01':
            newpixel = '001'
        elif pixel == '10':
            newpixel = '011'
        else:
            newpixel = '111'
        scanchain_map_final.append(newpixel)
    return(''.join(scanchain_map_final))